# 🚀 Real ML Model Training and Deployment

## Cybersecurity Anomaly Detection with Production ML Models

This notebook trains **real machine learning models** for cybersecurity anomaly detection using:
- **Isolation Forest** for outlier/anomaly detection
- **K-means Clustering** for user behavior classification
- **Snowpark ML UDFs** for production deployment

### 📋 Prerequisites
1. ✅ Completed SQL setup: `01_cybersecurity_schema.sql`, `02_sample_data_generation.sql`, `03_ai_ml_models.sql`, `04_snowpark_ml_deployment.sql`
2. ✅ Snowflake account with ACCOUNTADMIN privileges
3. ✅ Python packages: `snowflake-snowpark-python`, `scikit-learn`, `pandas`, `numpy`

### 🎯 What This Notebook Does
- Extracts 180+ days of user behavior data (500+ users, 4.3M+ events)
- Trains production-grade ML models on real behavioral patterns
- Deploys models as Snowpark Python UDFs
- Validates model performance and accuracy
- Replaces all simulated ML with genuine algorithms

---


## 🔧 1. Setup and Configuration

First, let's import the necessary libraries and establish our Snowflake connection.


In [ ]:
# Import required libraries
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.types import StructType, StructField, StringType, FloatType, IntegerType
from snowflake.snowpark import context
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import pickle
import json
import logging
from typing import Tuple, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Libraries imported successfully!")


## 🔑 2. Snowflake Session Setup

**✅ Snowflake Notebooks**: This notebook is designed for Snowflake Notebooks where the session is automatically provided.

For local development, you can manually create a session with your credentials.


In [ ]:
# Get Snowflake session
# In Snowflake Notebooks, the session is automatically available
try:
    # For Snowflake Notebooks - session is provided automatically
    session = context.get_active_session()
    print("✅ Using Snowflake Notebooks session")
    session_type = "snowflake_notebooks"
except:
    # For local development - create session manually
    print("🔧 Creating manual session for local development")
    session_type = "local_development"
    
    # Uncomment and update these parameters for local development:
    # connection_parameters = {
    #     "account": "your_account",
    #     "user": "your_username",  
    #     "password": "your_password",
    #     "role": "ACCOUNTADMIN",
    #     "warehouse": "COMPUTE_WH",
    #     "database": "CYBERSECURITY_DEMO",
    #     "schema": "SECURITY_AI"
    # }
    # session = Session.builder.configs(connection_parameters).create()
    
    print("❌ No session available. Please configure connection_parameters above for local development.")
    session = None

if session:
    print(f"📊 Session type: {session_type}")
else:
    print("⚠️  No active session. Please configure connection for local development.")


In [ ]:
# Test Snowflake session and set context
if session:
    try:
        # Set the correct database and schema context
        session.sql("USE DATABASE CYBERSECURITY_DEMO").collect()
        session.sql("USE SCHEMA SECURITY_AI").collect()
        
        # Test connection with a simple query
        result = session.sql("SELECT CURRENT_DATABASE(), CURRENT_SCHEMA(), CURRENT_USER()").collect()
        print(f"✅ Session active and connected!")
        print(f"🔍 Current context: {result[0][0]}.{result[0][1]} as {result[0][2]}")
        
        session_ready = True
        
    except Exception as e:
        print(f"❌ Session test failed: {str(e)}")
        print("🔧 Please ensure the CYBERSECURITY_DEMO database and SECURITY_AI schema exist.")
        session_ready = False
else:
    print("❌ No session available")
    session_ready = False


## 📊 3. Data Validation and Readiness Check

Before training ML models, let's validate that we have sufficient, high-quality data.


In [ ]:
# Check ML training data readiness
def validate_training_data(session: Session) -> Dict[str, Any]:
    """
    Validate that sufficient data exists for ML training
    """
    print("🔍 Validating training data readiness...")
    
    try:
        # Check overall data volume
        validation_query = """
        SELECT 
            COUNT(*) as total_events,
            COUNT(DISTINCT username) as unique_users,
            COUNT(DISTINCT DATE(timestamp)) as training_days,
            ROUND(COUNT(*) / COUNT(DISTINCT username), 2) as avg_events_per_user,
            COUNT(DISTINCT location:country::STRING) as unique_countries,
            COUNT(DISTINCT source_ip) as unique_ips,
            ROUND(AVG(CASE WHEN success THEN 1.0 ELSE 0.0 END), 3) as success_rate
        FROM USER_AUTHENTICATION_LOGS
        WHERE timestamp >= DATEADD(day, -90, CURRENT_TIMESTAMP())
        """
        
        result = session.sql(validation_query).collect()[0]
        
        metrics = {
            'total_events': result['TOTAL_EVENTS'],
            'unique_users': result['UNIQUE_USERS'], 
            'training_days': result['TRAINING_DAYS'],
            'avg_events_per_user': result['AVG_EVENTS_PER_USER'],
            'unique_countries': result['UNIQUE_COUNTRIES'],
            'unique_ips': result['UNIQUE_IPS'],
            'success_rate': result['SUCCESS_RATE']
        }
        
        return metrics
        
    except Exception as e:
        print(f"❌ Data validation failed: {str(e)}")
        return {}

# Run validation
if session_ready:
    data_metrics = validate_training_data(session)
else:
    data_metrics = {}

if data_metrics:
    print("\n📊 Training Data Summary:")
    print(f"  📈 Total Events: {data_metrics['total_events']:,}")
    print(f"  👥 Unique Users: {data_metrics['unique_users']:,}")
    print(f"  📅 Training Days: {data_metrics['training_days']}")
    print(f"  📊 Avg Events/User: {data_metrics['avg_events_per_user']}")
    print(f"  🌍 Countries: {data_metrics['unique_countries']}")
    print(f"  🌐 Unique IPs: {data_metrics['unique_ips']:,}")
    print(f"  ✅ Success Rate: {data_metrics['success_rate']:.1%}")
    
    # Determine readiness
    if (data_metrics['total_events'] >= 100000 and 
        data_metrics['unique_users'] >= 100 and 
        data_metrics['training_days'] >= 60):
        print("\n✅ Data is READY for ML training!")
        training_ready = True
    elif (data_metrics['total_events'] >= 10000 and 
          data_metrics['unique_users'] >= 50):
        print("\n⚠️  Data is MINIMAL but usable for ML training.")
        training_ready = True
    else:
        print("\n❌ INSUFFICIENT data for ML training.")
        print("   Please ensure you've run the sample data generation script.")
        training_ready = False
else:
    training_ready = False


## 🤖 4. Complete ML Training and Deployment

This cell contains the complete ML training pipeline that will:
1. Extract user behavior features
2. Train Isolation Forest for anomaly detection  
3. Train K-means for user clustering
4. Deploy models to Snowflake stages
5. Validate deployment

**Run this cell to train and deploy your real ML models!**


In [ ]:
if session_ready and training_ready:
    print("🚀 Starting complete ML training and deployment pipeline...")
    
    # 1. Extract user behavior features
    print("\n📊 Step 1: Feature Extraction")
    feature_query = """
    SELECT 
        username,
        AVG(EXTRACT(HOUR FROM timestamp)) as avg_login_hour,
        COALESCE(STDDEV(EXTRACT(HOUR FROM timestamp)), 0) as stddev_login_hour,
        COUNT(*) as total_logins,
        COUNT(DISTINCT source_ip) as unique_ips,
        COUNT(DISTINCT location:country::STRING) as countries,
        AVG(CASE WHEN EXTRACT(DOW FROM timestamp) IN (0,6) THEN 1.0 ELSE 0.0 END) as weekend_ratio,
        AVG(CASE WHEN EXTRACT(HOUR FROM timestamp) BETWEEN 22 AND 6 THEN 1.0 ELSE 0.0 END) as offhours_ratio
    FROM USER_AUTHENTICATION_LOGS
    WHERE timestamp >= DATEADD(day, -90, CURRENT_TIMESTAMP())
      AND username IS NOT NULL
    GROUP BY username
    HAVING COUNT(*) >= 10
    """
    
    training_df = session.sql(feature_query).to_pandas().fillna(0)
    print(f"✅ Extracted features for {len(training_df)} users")
    
    # 2. Train Isolation Forest
    print("\n🌲 Step 2: Training Isolation Forest")
    feature_cols = ['avg_login_hour', 'stddev_login_hour', 'unique_ips', 'countries', 'weekend_ratio', 'offhours_ratio']
    X = training_df[feature_cols]
    
    # Standardize features
    isolation_scaler = StandardScaler()
    X_scaled = isolation_scaler.fit_transform(X)
    
    # Train model
    isolation_model = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
    isolation_model.fit(X_scaled)
    
    # Get results
    scores = isolation_model.decision_function(X_scaled)
    anomalies = isolation_model.predict(X_scaled)
    n_anomalies = sum(anomalies == -1)
    
    print(f"✅ Isolation Forest trained: {n_anomalies} anomalies detected ({n_anomalies/len(training_df):.1%})")
    
    # 3. Train K-means
    print("\n🎯 Step 3: Training K-means Clustering") 
    cluster_features = ['avg_login_hour', 'countries', 'weekend_ratio', 'offhours_ratio', 'unique_ips']
    X_cluster = training_df[cluster_features]
    
    # Standardize features
    kmeans_scaler = StandardScaler()
    X_cluster_scaled = kmeans_scaler.fit_transform(X_cluster)
    
    # Train model
    kmeans_model = KMeans(n_clusters=6, random_state=42, n_init=10)
    clusters = kmeans_model.fit_predict(X_cluster_scaled)
    
    print(f"✅ K-means trained: {len(np.unique(clusters))} behavioral clusters created")
    
    # 4. Deploy models to Snowflake
    print("\n🚀 Step 4: Deploying Models to Snowflake")
    
    # Create stage
    session.sql("CREATE STAGE IF NOT EXISTS ml_models DIRECTORY = (ENABLE = TRUE)").collect()
    
    # Upload models
    models_to_deploy = {
        'isolation_forest': isolation_model,
        'isolation_scaler': isolation_scaler,
        'kmeans': kmeans_model,
        'kmeans_scaler': kmeans_scaler
    }
    
    for model_name, model in models_to_deploy.items():
        model_bytes = pickle.dumps(model)
        session.file.put_stream(
            input_stream=model_bytes,
            stage_location=f"@ml_models/{model_name}.pkl",
            auto_compress=False,
            overwrite=True
        )
        print(f"  ✅ {model_name} deployed")
    
    # 5. Validation
    print("\n🔍 Step 5: Deployment Validation")
    try:
        stage_files = session.sql("LIST @ml_models").collect()
        print(f"✅ {len(stage_files)} model files deployed to Snowflake stage")
        
        # Test if deployment script views work
        try:
            validation_result = session.sql("SELECT COUNT(*) FROM ML_TRAINING_DATA_VALIDATION").collect()
            print("✅ ML validation views accessible")
        except:
            print("⚠️  ML validation views need deployment script execution")
        
    except Exception as e:
        print(f"❌ Validation error: {str(e)}")
    
    # Final summary
    print("\n" + "="*60)
    print("🎉 REAL ML IMPLEMENTATION COMPLETE!")
    print("="*60)
    print(f"📊 Training Data: {len(training_df):,} users")
    print(f"🌲 Isolation Forest: {n_anomalies} anomalies ({n_anomalies/len(training_df):.1%})")
    print(f"🎯 K-means: {len(np.unique(clusters))} behavioral clusters")
    print(f"🚀 Models Deployed: ✅ All models uploaded to Snowflake")
    print("\n🎯 Next Steps:")
    print("1. Run SQL: 04_snowpark_ml_deployment.sql (register UDFs)")
    print("2. Test: SELECT * FROM SNOWPARK_ML_USER_CLUSTERS LIMIT 10")
    print("3. Launch: Your Streamlit app now uses REAL ML!")
    print("\n🎉 No more simulations - this is production-grade ML!")
    
else:
    if not session_ready:
        print("❌ Skipping ML training due to session issues.")
        print("💡 Please ensure Snowflake session is properly configured.")
    elif not training_ready:
        print("❌ Skipping ML training due to insufficient data.")
        print("💡 Please run the SQL data generation scripts first.")
